### **块 1: 环境设置、库导入与路径管理**

**目标:** 初始化项目环境，加载所有必需的库，并根据 `RUNNING_ENV` 和 `TEST_MODE` 智能配置所有输入输出文件的路径。此代码块是整个Notebook的控制中心，负责确保代码的可移植性、可重复性和易于调试。

In [1]:
# =============================================================================
# --- 块 1: 环境设置、库导入与路径管理 ---
# =============================================================================

# 导入必要的库
import pandas as pd
import os
import pickle
import time
from tqdm.auto import tqdm
import psutil
import shutil
from collections import Counter
import gc # 垃圾回收库，用于主动释放内存

# --- 核心配置区 ---
# 确保这里的配置与你的项目环境一致
RUNNING_ENV = 'local'  # 可选: 'local' 或 'dsw'
TEST_MODE = True       # 可选: True 或 False
# 如果不是测试模式，可以处理更多候选，但人工审核工作量会增加
MAX_CANDIDATES_FOR_REVIEW = 5000
MIN_FREQUENCY_THRESHOLD = 5 if TEST_MODE else 10

# --- 路径智能管理 ---
print(f"检测到运行环境为: 【{RUNNING_ENV.upper()}】")
TEMP_DIR = '/tmp'

if RUNNING_ENV == 'local':
    print("使用 'local' 模式的相对路径。")
    BASE_DATA_PROCESSED_PATH = '../data_processed'
elif RUNNING_ENV == 'dsw':
    print("使用 'dsw' 模式的绝对路径。")
    BASE_DATA_PROCESSED_PATH = '/mnt/data/data_processed'
    if not os.path.exists(TEMP_DIR): os.makedirs(TEMP_DIR)
else:
    raise ValueError(f"未知的 RUNNING_ENV: '{RUNNING_ENV}'. 请选择 'local' 或 'dsw'。")

# --- 定义所有“原始”文件路径 ---
# 输入文件
TOKEN_LISTS_ORIGINAL = os.path.join(BASE_DATA_PROCESSED_PATH, 'intermediate_raw_token_lists.pkl')
CANDIDATES_GENSIM_ORIGINAL = os.path.join(BASE_DATA_PROCESSED_PATH, 'candidates_gensim.pkl')
CANDIDATES_NER_ORIGINAL = os.path.join(BASE_DATA_PROCESSED_PATH, 'candidates_ner.pkl')
# 输出文件
CANDIDATES_FOR_REVIEW_ORIGINAL = os.path.join(BASE_DATA_PROCESSED_PATH, 'candidate_phrases_for_review.csv')

# --- 初始化实际使用的路径变量 ---
TOKEN_LISTS_PATH = TOKEN_LISTS_ORIGINAL
CANDIDATES_GENSIM_PATH = CANDIDATES_GENSIM_ORIGINAL
CANDIDATES_NER_PATH = CANDIDATES_NER_ORIGINAL
CANDIDATES_FOR_REVIEW_PATH = CANDIDATES_FOR_REVIEW_ORIGINAL

# --- DSW环境下的I/O优化 ---
if RUNNING_ENV == 'dsw':
    print("DSW 环境模式已激活，将智能检查并使用本地临时目录 /tmp ...")
    # 定义临时路径
    TEMP_TOKEN_LISTS = os.path.join(TEMP_DIR, 'intermediate_raw_token_lists.pkl')
    TEMP_CANDIDATES_GENSIM = os.path.join(TEMP_DIR, 'candidates_gensim.pkl')
    TEMP_CANDIDATES_NER = os.path.join(TEMP_DIR, 'candidates_ner.pkl')
    TEMP_CANDIDATES_FOR_REVIEW = os.path.join(TEMP_DIR, 'candidate_phrases_for_review.csv')

    # 智能复制函数
    def sync_to_tmp(original_path, temp_path):
        if os.path.exists(original_path):
            if not os.path.exists(temp_path) or os.path.getsize(original_path) != os.path.getsize(temp_path):
                print(f"正在从 {original_path} 同步到 {temp_path}...")
                shutil.copy(original_path, temp_path)
                print("同步完成。")
            else:
                 print(f"临时文件 {temp_path} 已是最新，跳过同步。")
            return temp_path
        else:
            print(f"⚠️ 警告: 源文件 {original_path} 不存在。")
            return None

    # 对所有输入文件执行同步并重定向路径
    TOKEN_LISTS_PATH = sync_to_tmp(TOKEN_LISTS_ORIGINAL, TEMP_TOKEN_LISTS)
    CANDIDATES_GENSIM_PATH = sync_to_tmp(CANDIDATES_GENSIM_ORIGINAL, TEMP_CANDIDATES_GENSIM)
    CANDIDATES_NER_PATH = sync_to_tmp(CANDIDATES_NER_ORIGINAL, TEMP_CANDIDATES_NER)
    CANDIDATES_FOR_REVIEW_PATH = TEMP_CANDIDATES_FOR_REVIEW # 输出路径直接指向/tmp

# --- 打印最终配置信息 ---
print("\n--- 环境准备 ---")
if TEST_MODE:
    print(f"🚀🚀🚀 运行在【快速测试模式】下！🚀🚀🚀")
else:
    print(f"🚢🚢🚢 运行在【完整数据模式】下。🚢🚢🚢")

print(f"Gensim候选输入: {CANDIDATES_GENSIM_PATH}")
print(f"NER候选输入: {CANDIDATES_NER_PATH}")
print(f"Token列表输入 (用于计算频率): {TOKEN_LISTS_PATH}")
print(f"待审核文件输出: {CANDIDATES_FOR_REVIEW_PATH}")
print(f"最低频率阈值: {MIN_FREQUENCY_THRESHOLD}")

检测到运行环境为: 【LOCAL】
使用 'local' 模式的相对路径。

--- 环境准备 ---
🚀🚀🚀 运行在【快速测试模式】下！🚀🚀🚀
Gensim候选输入: ../data_processed\candidates_gensim.pkl
NER候选输入: ../data_processed\candidates_ner.pkl
Token列表输入 (用于计算频率): ../data_processed\intermediate_raw_token_lists.pkl
待审核文件输出: ../data_processed\candidate_phrases_for_review.csv
最低频率阈值: 5


### **块 2: 加载候选短语与Token列表**

**目标:** 将上一流程（`02_Entity_Phrase_Solidification.ipynb`）生成的两个候选短语池（Gensim和NER）以及用于计数的原始Token列表从磁盘加载到内存中。此步骤是后续所有整合工作的数据基础。

In [2]:
# =============================================================================
# --- 块 2: 加载候选短语与Token列表 ---
# =============================================================================

print("--- 阶段 2.1: 加载候选短语与Token列表 ---")
start_time = time.time()

# 初始化为空，以防文件加载失败
gensim_candidates, ner_candidates, raw_token_lists = {}, {}, []

try:
    with open(CANDIDATES_GENSIM_PATH, 'rb') as f:
        gensim_candidates = pickle.load(f)
    print(f"✅ 成功加载 {len(gensim_candidates)} 个 Gensim 候选短语。")

    with open(CANDIDATES_NER_PATH, 'rb') as f:
        ner_candidates = pickle.load(f)
    print(f"✅ 成功加载 {len(ner_candidates)} 个 NER 候选实体。")

    with open(TOKEN_LISTS_PATH, 'rb') as f:
        raw_token_lists = pickle.load(f)
    print(f"✅ 成功加载 {len(raw_token_lists)} 篇文章的Token列表。")

except FileNotFoundError as e:
    print(f"❌ 错误: 缺少必要的输入文件: {e.filename}。")
    print("请确保已成功运行 '02_Entity_Phrase_Solidification.ipynb'。")

print(f"加载耗时: {time.time() - start_time:.2f} 秒。")

--- 阶段 2.1: 加载候选短语与Token列表 ---
✅ 成功加载 3172 个 Gensim 候选短语。
✅ 成功加载 7920 个 NER 候选实体。
✅ 成功加载 1000 篇文章的Token列表。
加载耗时: 0.06 秒。


### **块 3: 构建与合并数据框 (含频率计算优化)**

**目标:** 将两个来源的候选短语统一到单个Pandas DataFrame中，并高效地为Gensim发现的短语计算其在整个语料库中的出现频率。

**优化点:** 此处采用了Qwen建议的优化方案。我们不再使用效率较低的字符串`.count()`方法，而是通过**预处理候选短语**（按长度分组）和**滑动窗口遍历Token列表**的方式，显著提升频率计算的性能。

In [3]:
# =============================================================================
# --- 块 3: 构建与合并数据框 (含频率计算优化) ---
# =============================================================================

if gensim_candidates and ner_candidates:
    print("\n--- 阶段 2.2: 构建与合并数据框 ---")
    start_time = time.time()

    # 3.1 处理Gensim数据
    print("正在处理 Gensim 候选...")
    df_gensim = pd.DataFrame(gensim_candidates.items(), columns=['candidate_phrase', 'score'])
    df_gensim['source'] = 'Gensim'
    print(f"  - Gensim DataFrame 创建完成，形状: {df_gensim.shape}")

    # 3.2 处理NER数据
    print("正在处理 NER 候选...")
    df_ner = pd.DataFrame(ner_candidates.items(), columns=['candidate_phrase', 'frequency'])
    df_ner['source'] = 'NER'
    print(f"  - NER DataFrame 创建完成，形状: {df_ner.shape}")

    # 3.3 [优化版] 高效计算Gensim短语频率
    print("正在为 Gensim 短语高效计算频率 (优化版)...")

    # 预处理：将短语按其包含的token数量分组
    gensim_phrases_by_len = {}
    for phrase_str in df_gensim['candidate_phrase']:
        tokens = phrase_str.split()
        n = len(tokens)
        if n > 1: # 只处理多词短语
            if n not in gensim_phrases_by_len:
                gensim_phrases_by_len[n] = set()
            gensim_phrases_by_len[n].add(tuple(tokens)) # 使用元组以支持哈希

    phrase_counts = Counter()

    # 遍历所有文章的token列表
    for doc_tokens_case_sensitive in tqdm(raw_token_lists, desc="高效计算Gensim频率"):
        doc_tokens = [token.lower() for token in doc_tokens_case_sensitive]
        doc_len = len(doc_tokens)

        # 对每种可能的短语长度进行滑动窗口检查
        for n, phrases_set in gensim_phrases_by_len.items():
            if doc_len < n:
                continue
            for i in range(doc_len - n + 1):
                window_tuple = tuple(doc_tokens[i : i + n])
                if window_tuple in phrases_set:
                    phrase_counts[" ".join(window_tuple)] += 1

    # 将计算出的频率映射回df_gensim
    df_gensim['frequency'] = df_gensim['candidate_phrase'].map(phrase_counts)
    df_gensim['frequency'] = df_gensim['frequency'].fillna(0).astype(int)
    print("  - Gensim 频率计算完成。")

    # 释放内存
    del raw_token_lists, phrase_counts, gensim_phrases_by_len
    gc.collect()

    # 3.4 合并
    print("正在合并两个来源的数据框...")
    df_candidates = pd.concat([df_ner, df_gensim], ignore_index=True)
    print(f"  - 合并完成。总候选数: {len(df_candidates)}")
    print(f"构建与合并耗时: {time.time() - start_time:.2f} 秒。")
else:
    print("候选数据未加载，跳过此块。")


--- 阶段 2.2: 构建与合并数据框 ---
正在处理 Gensim 候选...
  - Gensim DataFrame 创建完成，形状: (3172, 3)
正在处理 NER 候选...
  - NER DataFrame 创建完成，形状: (7920, 3)
正在为 Gensim 短语高效计算频率 (优化版)...


高效计算Gensim频率:   0%|          | 0/1000 [00:00<?, ?it/s]

  - Gensim 频率计算完成。
正在合并两个来源的数据框...
  - 合并完成。总候选数: 11092
构建与合并耗时: 0.35 秒。


### **块 4: 清洗、过滤、排序与去重**

**目标:** 对合并后的候选池进行“提纯”，移除明显无用的条目（如低频、含数字），并根据`NER优先`的原则进行去重，最后按`频率`降序排列，生成一份整洁、有序、待人工审核的列表。

In [4]:
# =============================================================================
# --- 块 4: 清洗、过滤、排序与去重 ---
# =============================================================================

if 'df_candidates' in locals() and not df_candidates.empty:
    print("\n--- 阶段 2.3: 清洗、过滤、排序与去重 ---")
    start_time = time.time()

    # 4.1 清洗与标准化
    print(f"原始候选数: {len(df_candidates)}")
    df_candidates['candidate_phrase'] = df_candidates['candidate_phrase'].str.strip()
    df_candidates.dropna(subset=['candidate_phrase', 'frequency'], inplace=True)
    df_candidates = df_candidates[df_candidates['candidate_phrase'] != '']
    # 过滤掉包含数字的短语和过短的短语
    df_candidates = df_candidates[~df_candidates['candidate_phrase'].str.contains(r'\d')]
    df_candidates = df_candidates[df_candidates['candidate_phrase'].str.len() > 3]
    print(f"清洗后候选数: {len(df_candidates)}")

    # 4.2 过滤低频项
    df_candidates = df_candidates[df_candidates['frequency'] >= MIN_FREQUENCY_THRESHOLD]
    print(f"过滤低频项后 (频率 >= {MIN_FREQUENCY_THRESHOLD}): {len(df_candidates)}")

    # 4.3 去重与优先级处理 (核心逻辑)
    # 将'source'列转为有序的Categorical类型，以定义优先级
    df_candidates['source'] = pd.Categorical(df_candidates['source'], categories=['Gensim', 'NER'], ordered=True)
    # 按 候选短语 分组，按 source 优先级排序 (NER在后，所以是更高优先级)
    df_candidates.sort_values(['candidate_phrase', 'source'], ascending=[True, True], inplace=True)
    # 去重，保留最后一个（即优先级最高的NER）
    df_candidates.drop_duplicates(subset=['candidate_phrase'], keep='last', inplace=True)
    print(f"去重后 (NER优先): {len(df_candidates)}")

    # 4.4 最终排序
    df_candidates.sort_values(by='frequency', ascending=False, inplace=True)
    print("已按频率降序排列。")

    # 如果不是测试模式，则截取前 MAX_CANDIDATES_FOR_REVIEW 个
    if not TEST_MODE and len(df_candidates) > MAX_CANDIDATES_FOR_REVIEW:
        df_candidates = df_candidates.head(MAX_CANDIDATES_FOR_REVIEW)
        print(f"已截取频率最高的 {MAX_CANDIDATES_FOR_REVIEW} 个候选进行审核。")

    # 4.5 添加审核列
    df_candidates['action_code'] = ''
    df_candidates['standard_form'] = ''

    # 调整列顺序以便审核
    final_columns = ['candidate_phrase', 'frequency', 'source', 'score', 'action_code', 'standard_form']
    df_final_review = df_candidates.reindex(columns=final_columns)

    print(f"最终待审核候选数: {len(df_final_review)}")
    print(f"清洗与排序耗时: {time.time() - start_time:.2f} 秒。")

else:
    print("候选DataFrame未创建，跳过此块。")


--- 阶段 2.3: 清洗、过滤、排序与去重 ---
原始候选数: 11092
清洗后候选数: 10779
过滤低频项后 (频率 >= 5): 3364
去重后 (NER优先): 3142
已按频率降序排列。
最终待审核候选数: 3142
清洗与排序耗时: 0.02 秒。


### **块 5: 保存待审核文件**

**目标:** 将最终处理完成的、待人工审核的DataFrame保存为CSV文件。同时，根据运行环境，负责将结果从临时目录同步回项目的数据目录，并给出清晰的下一步操作指引。

In [5]:
# =============================================================================
# --- 块 5: 保存待审核文件 ---
# =============================================================================

if 'df_final_review' in locals() and not df_final_review.empty:
    print("\n--- 阶段 2.4: 保存待审核文件 ---")
    try:
        # 修正FutureWarning
        df_final_review['frequency'] = df_final_review['frequency'].fillna(0).astype(int)

        df_final_review.to_csv(CANDIDATES_FOR_REVIEW_PATH, index=False, encoding='utf-8-sig') # 使用 utf-8-sig 确保Excel能正确打开

        final_output_path = CANDIDATES_FOR_REVIEW_PATH
        # 如果是DSW环境，将结果从/tmp同步回网络存储
        if RUNNING_ENV == 'dsw':
            print(f"正在从 {CANDIDATES_FOR_REVIEW_PATH} 同步回 {CANDIDATES_FOR_REVIEW_ORIGINAL}...")
            shutil.copy(CANDIDATES_FOR_REVIEW_PATH, CANDIDATES_FOR_REVIEW_ORIGINAL)
            final_output_path = CANDIDATES_FOR_REVIEW_ORIGINAL
            print("同步完成。")

        print(f"\n✅✅✅ 成功生成待审核文件！ ✅✅✅")
        print(f"文件路径: {final_output_path}")
        print(f"包含 {len(df_final_review)} 条待审核的候选短语。")
        print("\n下一步操作:")
        print("1. 打开这个CSV文件 (推荐使用Excel或Google Sheets)。")
        print("2. 根据需求文档的指引，在 'action_code' 和 'standard_form' 列填写您的决策。")
        print("3. 保存修改后的文件，为下一阶段 '规则生成与应用' 做准备。")

    except Exception as e:
        print(f"❌ 保存文件时发生错误: {e}")
else:
    print("没有可供保存的待审核数据。")


--- 阶段 2.4: 保存待审核文件 ---

✅✅✅ 成功生成待审核文件！ ✅✅✅
文件路径: ../data_processed\candidate_phrases_for_review.csv
包含 3142 条待审核的候选短语。

下一步操作:
1. 打开这个CSV文件 (推荐使用Excel或Google Sheets)。
2. 根据需求文档的指引，在 'action_code' 和 'standard_form' 列填写您的决策。
3. 保存修改后的文件，为下一阶段 '规则生成与应用' 做准备。
